In [2]:
import pandas as pd
import numpy as np

In [11]:
#rhomis_data = pd.read_csv("/Users/anab/Documents/MS_UCDavis/STA208/project/RHoMIS_Full_Data.csv", engine = "python")
#rhomis_data.head()

In [17]:
class Functions:
    """
     Conatins all cleaning function for this data
    """
    
#     def __init__(self,df):

    def getNAs(self, df):
        colNames = []
        percentNA = []
        for i in df.columns:
            colNames.append(i)
            numNA = df[i].isna().sum()
            percent = (numNA/len(df))*100
            percentNA.append(percent)

        colNames = pd.DataFrame(colNames)
        colNames = colNames.rename(columns={0: "label"})
        percentNA = pd.DataFrame(percentNA)
        percentNA = percentNA.rename(columns={0: "numNA"})
        self.data = pd.concat([colNames,percentNA], axis = 1).sort_values(by=['numNA'], ascending = False)

        return self.data
        
    def drop_columns(self, df, threshold_percent):
        '''drop columns by a threshold (percentage of na)'''
        global data ### double check...
        data = self.getNAs(df)
        column_names = data[data.numNA >= threshold_percent].label
        self.clean_data = df.drop(column_names, axis = 1)

        return self.clean_data
    
    def entry_to_lowercase(self, df):
        for i in df.columns:
            if (df[i].dtype == "O"):
                df[i] = df[i].str.lower()
        return df
    
    def id_data_types(self, df):
        names = pd.DataFrame()
        category = pd.DataFrame()
        data_type = pd.DataFrame()
        numUni = pd.DataFrame()

        for i in df.columns:

            names = names.append({"variable": i},ignore_index = True)
            category = category.append({"category": list(df[i].unique())}, ignore_index = True)
            numUni = numUni.append({"numUnique": len(df[i].unique())}, ignore_index = True)
            data_type = data_type.append({"data_type": df[i].dtype}, ignore_index = True)

        view_data = pd.concat([names, category, numUni,data_type],axis =1)

        return (view_data)
    
    def replace_na_with_NaN(self, df):
        
        """
         Input: Full dataset
         Output: Full dataset where object strings have underscores and spaces removed
         EX:'No_School-> NoSchool, no school -> noschool'
        """
        
        for i in df.columns:
            df[i] = df[i].replace(np.nan, 'na')
        return df
    
    # removes underscores and spaces from instances, 
    def remove_underscores_spaces(self, df):
        
        """
         Input: Full dataset
         Output: Full dataset where object strings have underscores and spaces removed
         EX:'No_School-> NoSchool, no school -> noschool'
        """
        
        for i in df.columns:
            if (df[i].dtype == "O"):
                df[i] = df[i].str.replace('_', "")
                df[i] = df[i].str.replace(' ', "")
        return df
    
    
    
    def convert_to_categorical(self, df, variables):
        """
         Input: dataframe, column names of variables to be transformed
         Output: dataframe with dtypes of indicated variables changed
         
         #NOTE: USE this function AFTER preprocessing functions above
        """
    
    # convert these to categories
    #df[v1] = df[v1].astype(‘category’)
    
        for i in variables:
            df[i] = df[i].astype('category')
        return df
    
    
    def impute_data(self, Xtrain, Xtest):
        """
        Input: XTraining set, XTesting Set
        Output: The inputs with NA numerical variabes imputed by column median, and categorical
        NA varaibles converted into a seperate category "Na" 
        """
        
       
   
        num_train = Xtrain.select_dtypes(include=['float64', 'int64'])
        num_test = Xtest.select_dtypes(include=['float64', 'int64'])
        
        cat_train = Xtrain.select_dtypes(include=['object', 'category'])
        cat_test = Xtest.select_dtypes(include=['object', 'category'])
      

    # imputes NA numerics with median
        for i in num_train.columns:
            Xtrain[i] = num_train.fillna(np.nanmedian(Xtrain[i]))
        
        for i in num_test.columns:
            Xtest[i] = num_test.fillna(np.nanmedian(num_test[i]))
        
    # categorical NA to "Na" level
        for i in cat_train.columns:
            Xtrain[i] = cat_train.replace(np.nan, "Na")
        
        for i in cat_test.columns:
            Xtest[i] = cat_test.replace(np.nan, "Na")

   
        return Xtrain, Xtest

    def drop_response_rows_with_NAs(self,df,y_variable):
        """
        Input: dataframe, response variable column 
        Output: dataframe rows with missing response varaibles
        are dropped entirely (we cannot use these to train or test)
        But these dropped rows are saved to be used later
        
        """
    
    ## need to delete PPI_threshold since that is like our y2 which we won’t use
        df = df.drop("PPI_Threshold", axis=1)
        condition = df[y_variable]
        rows_to_delete = df[np.isnan(condition)].index
    
    #save all rows with y=NA. Will use this data to predict the y later.
        prediction_dataset = df.iloc[rows_to_delete,:]
    
    #create new data
        new_data = df.drop(labels=rows_to_delete, axis=0)
    
        return new_data, prediction_dataset


In [3]:
dat = pd.read_csv("../data/clean.csv")
from Functions import Cleaning_Functions
fun = Cleaning_Functions()
#fun.id_data_types(dat)

In [4]:
#check if replaceNan function works with updated .py file
b= fun.replace_na_with_NaN(dat)
b.head()

,crop_count,crop_name_1,crop_harvest_1,crop_intercrop_1,ID_PROJ,ID_COUNTRY,YEAR,SURVEY_ID,ID_HH,Country,...,value_livestock_production_USD_PPP_pHH_Yr,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Market_Orientation,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,Gender_MaleControl,Gender_FemaleControl,continent
0,na,groundnut,na,na,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_1_1,Tanzania,...,38.500640,0.000000,1,0.049505,3650.57,0,0,0,1,africa
1,na,millet,na,na,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_2_1,Tanzania,...,0.000000,0.000000,1,0,3632.45,0,4,0.499999,0.500001,africa
2,na,groundnut,na,na,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_3_1,Tanzania,...,128.335467,0.000000,1,0.12987,5314.78,1750,1,1,0,africa
3,na,cowpea,na,na,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_4_1,Tanzania,...,0.000000,0.000000,na,na,257.235,257.235,0,na,na,africa
4,na,sesame,na,na,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_5_1,Tanzania,...,233.994961,23.102694,0.931377,0.695041,925.31,22.0702,0,1,0,africa


In [11]:
fun.replace_na_with_NaN(dat)
fun.entry_to_lowercase(dat)
fun.remove_underscores_spaces(dat)
fun.convert_to_categorical(dat,["crop_harvest_1", "crop_intercrop_1", "YEAR", "HFIAS_status", "continent"])
dat, s = fun.drop_response_rows_with_NAs(dat, "Market_Orientation")

In [5]:
from sklearn.model_selection import train_test_split

X = dat.loc[:, dat.columns != 'Market_Orientation']
y = dat.loc[:, "Market_Orientation"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = .3)

In [6]:
X_train, X_test= fun.impute_data(X_train, X_test)

C:\Users\chris\Desktop\Sta 208\Project\STA_208\finalized_code\Functions.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  Xtrain[i] = num_train.fillna(np.nanmedian(Xtrain[i]))
C:\Users\chris\Desktop\Sta 208\Project\STA_208\finalized_code\Functions.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  Xtest[i] = num_test.fillna(np.nanmedian(num_test[i]))
C:\Users\chris\Desktop\Sta 208\Project\STA_208\finalized_code\Functions.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [7]:
X_train

,crop_count,crop_name_1,crop_harvest_1,crop_intercrop_1,ID_PROJ,ID_COUNTRY,YEAR,SURVEY_ID,ID_HH,Country,...,livestock_prodsales_USD_PPP_pHH_Yr,value_livestock_production_USD_PPP_pHH_Yr,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,Gender_MaleControl,Gender_FemaleControl,continent
10955,1.0,maize,maize,maize,maize,maize,maize,maize,maize,maize,...,1.000000,1.000000,1.0,1.000000,1.00000,1.0,1.0,1.000000,1.000000,maize
13231,1.0,maize,maize,maize,maize,maize,maize,maize,maize,maize,...,1.000000,1.000000,1.0,1.000000,1.00000,1.0,1.0,1.000000,1.000000,maize
8475,1.0,rice,rice,rice,rice,rice,rice,rice,rice,rice,...,1.000000,1.000000,1.0,1.000000,1.00000,1.0,1.0,1.000000,1.000000,rice
3974,2.0,cassava,cassava,cassava,cassava,cassava,cassava,cassava,cassava,cassava,...,2.000000,2.000000,2.0,2.000000,2.00000,2.0,2.0,2.000000,2.000000,cassava
1168,1.0,millet,millet,millet,millet,millet,millet,millet,millet,millet,...,1.000000,1.000000,1.0,1.000000,1.00000,1.0,1.0,1.000000,1.000000,millet
7960,3.0,maize,maize,maize,maize,maize,maize,maize,maize,maize,...,3.000000,3.000000,3.0,3.000000,3.00000,3.0,3.0,3.000000,3.000000,maize
8696,3.0,maize,maize,maize,maize,maize,maize,maize,maize,maize,...,3.000000,3.000000,3.0,3.000000,3.00000,3.0,3.0,3.000000,3.000000,maize
2692,1.0,sorghum,sorghum,sorghum,sorghum,sorghum,sorghum,sorghum,sorghum,sorghum,...,1.000000,1.000000,1.0,1.000000,1.00000,1.0,1.0,1.000000,1.000000,sorghum
5495,1.0,teff,teff,teff,teff,teff,teff,teff,teff,teff,...,1.000000,1.000000,1.0,1.000000,1.00000,1.0,1.0,1.000000,1.000000,teff
1263,3.0,millet,millet,millet,millet,millet,millet,millet,millet,millet,...,3.000000,3.000000,3.0,3.000000,3.00000,3.0,3.0,3.000000,3.000000,millet
